# UK COVID-19 Dashboard

### A comparison of occupied mechanical ventilator beds and new cases with first and second vaccine doses

[Source code](https://github.com/jmdwrntn/ukcovid19dashboard) (C) James Thornton, 2021 ([jmdwrntn@mailbox.org](mailto:jmdwrntn@mailbox.org)). All rights reserved.
 *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

In [123]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [127]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [128]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("beds.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [129]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [130]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])   

    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['beds', 'cases', 'firstDose', 'secondDose'])

    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        parsed_date=parse_date(entry['date'])
        for column in ['beds', 'cases', 'firstDose', 'secondDose']:
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
            if pd.isna(df.loc[parsed_date, column]): 
            # replace None with 0 in our data 
                value=float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
                df.loc[parsed_date, column]=value
            
    # fill in any remaining "holes" due to missing dates
    df.fillna(0.0, inplace=True)

    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
# df=wrangle_data(jsondata) # df is the dataframe for plotting
# df.plot(logy=True)

df=wrangle_data(jsondata)

In [131]:
filters = [
    "areaType=overview"
]

structure = {
    "date": "date",
    "beds": "covidOccupiedMVBeds",
    "cases": "newCasesByPublishDate",
    "firstDose": "newPeopleVaccinatedFirstDoseByPublishDate",
    "secondDose": "newPeopleVaccinatedSecondDoseByPublishDate"
}

latestdata = Cov19API(filters=filters, structure=structure)

In [132]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    return latestdata.get_json() # return data read from the API

In [133]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    
    if len(apidata) > 0:
        button.icon="check-circle"
        button.description="Synced"
        button.disabled=True
        global df
        df=wrangle_data(apidata)
        refresh_graph()
    else:
        button.icon="exclamation-circle"
        button.description="Error"
        button.style="warning"
        button.tooltip="Could not sync latest data, falling back to current data"
    # wrangle the data and overwrite the dataframe for plotting
    
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    # refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    # apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Sync data', # you may want to change this...
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Sync latest data from PHE",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='cloud-download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets


# run all cells before clicking on this button

## Interactive graph

The graph below shows PHE COVID-19 data from 1 January to 30 November 2021 (when this dashboard was created). If this range is now out of date, please press the button below to refresh the dataset, after which the graph should update itself. If there is any issue requesting new data from PHE, the dashboard will fall back to using the original data.

The following metrics are shown in the graph:
* Mechanical ventilator beds occupied by COVID-19 patients (*'beds'*)[<sup>1</sup>](#fn1)
* New cases by publish date (*'cases'*)
* New people who received first vaccination dose by publish date (*'firstDose'*)
* New people who received first vaccination dose by publish date (*'secondDose'*)

As the numbers in the *'beds'* metric are significantly lower than the other three metrics, the data is displayed in a logarithmic scale. This also allows the representation of **exponential growth** within a small table, which be challenging with other forms of graph.

For example, the period of time on the graph immediately after January 2021 shows an exponential rise of first vaccine doses being administered (from zero, effectively). The **y** scale shows around 100,000 first vaccine doses were administered (10<sup>5</sup>), approximately 50,000 new cases were published (10<sup>4</sup>), and approximately 4000 mechnical ventilator beds were occupied by COVID-19 patients (10<sup>3</sup>).


##### <span id="fn1">1. It is assumed that the mechanical ventilator beds occupied by COVID patients metric is cumulative rather than new patients each day, as the [Developer's Guide](https://coronavirus.data.gov.uk/details/developers-guide) for the API does not provide detail.</span>

In [134]:
display(apibutton)

Button(description='Sync data', icon='cloud-download', style=ButtonStyle(), tooltip='Sync latest data from PHE…

In [138]:
bedcols=wdg.SelectMultiple(
    options=['beds', 'cases', 'firstDose', 'secondDose'], # options available
    value=['beds', 'cases', 'firstDose'], # initial value
    rows=4, # rows of the selection box
    description='Options',
    disabled=False
)

def refresh_graph():
    current=bedcols.value
    if current==bedcols.options[0:2]:
        other=bedcols.options[0:3]
    else:
        other=bedcols.options[0:2]
    bedcols.value=other
    bedcols.value=current

def beds_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        df.plot(logy=True, y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.title("UK COVID-19: Occupied Beds, Cases and Vaccinations by Dosage")
        plt.xlabel("Month")
        plt.ylabel("Number of people")
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling beds_graph(graphcolumns=value_of_bedcols); capture output in widget output    
output=wdg.interactive_output(beds_graph, {'graphcolumns': bedcols})

controls=wdg.VBox([apibutton, bedcols])
graph=wdg.HBox([output, controls])
display(graph)

#display(bedcols, output)